In [0]:
from pyspark.sql.functions import col, to_timestamp, lit, coalesce

# BRONZE paths (already exist)
bronze_atm_path   = "/mnt/bronze/ATMTransactions"
bronze_upi_path   = "/mnt/bronze/UPIEvents"
bronze_fraud_path = "/mnt/bronze/FraudAlerts"

# SILVER paths (we will create these now)
silver_atm_path   = "/mnt/silver/ATMTransactions"
silver_upi_path   = "/mnt/silver/UPIEvents"
silver_fraud_path = "/mnt/silver/FraudAlerts"


In [0]:
# Read from Bronze & inspect
atm_bronze = spark.read.format("delta").load(bronze_atm_path)

display(atm_bronze.limit(5))
atm_bronze.printSchema()


TransactionID,ATMID,AccountNumber,CustomerID,TransactionAmount,TransactionType,TransactionTime,Location,Status,id,txn_type,suspicious_flags,is_suspicious,_rid,_self,_etag,_attachments,_ts,Amount,processedAt,fraud_flags
ATM009778,ATM035,1002003360,CUST259,5000.0,WITHDRAWAL,2025-01-07T18:57:00Z,Kolkata,SUCCESS,768a7cf6-b831-40d7-84d2-6936df5c6352,ATM,None,0,jk1hAMNfqcwB1AAAAAAAAA==,dbs/jk1hAA==/colls/jk1hAMNfqcw=/docs/jk1hAMNfqcwB1AAAAAAAAA==/,"""000032d4-0000-5900-0000-6935ba710000""",attachments/,1.765128817E9,NaN,nan,null
ATM004496,ATM020,1002003320,CUST008,60000.0,WITHDRAWAL,2025-01-04T02:55:00Z,Pune,SUCCESS,a0d37bfd-df35-4f5c-ab9b-21d60d958b05,ATM,None,0,jk1hAMNfqcwC1AAAAAAAAA==,dbs/jk1hAA==/colls/jk1hAMNfqcw=/docs/jk1hAMNfqcwC1AAAAAAAAA==/,"""000033d4-0000-5900-0000-6935ba710000""",attachments/,1.765128817E9,NaN,nan,null
ATM004497,ATM050,1002003300,CUST183,45000.0,WITHDRAWAL,2025-01-04T02:56:00Z,Hyderabad,SUCCESS,825ba3ac-469b-46d8-88fa-a125bb27826d,ATM,None,0,jk1hAMNfqcwD1AAAAAAAAA==,dbs/jk1hAA==/colls/jk1hAMNfqcw=/docs/jk1hAMNfqcwD1AAAAAAAAA==/,"""000034d4-0000-5900-0000-6935ba710000""",attachments/,1.765128817E9,NaN,nan,null
ATM009779,ATM028,1002003496,CUST224,2000.0,WITHDRAWAL,2025-01-07T18:58:00Z,Hyderabad,SUCCESS,a0f30efb-b75b-427e-a914-26fac0bbcd36,ATM,None,0,jk1hAMNfqcwE1AAAAAAAAA==,dbs/jk1hAA==/colls/jk1hAMNfqcw=/docs/jk1hAMNfqcwE1AAAAAAAAA==/,"""000035d4-0000-5900-0000-6935ba710000""",attachments/,1.765128817E9,NaN,nan,null
ATM004498,ATM001,1002003185,CUST294,60000.0,WITHDRAWAL,2025-01-04T02:57:00Z,Kolkata,SUCCESS,afcbe4a5-dbb3-49e9-ac78-bd8568cc5f9b,ATM,None,0,jk1hAMNfqcwF1AAAAAAAAA==,dbs/jk1hAA==/colls/jk1hAMNfqcw=/docs/jk1hAMNfqcwF1AAAAAAAAA==/,"""000036d4-0000-5900-0000-6935ba710000""",attachments/,1.765128817E9,NaN,nan,null


root
 |-- TransactionID: string (nullable = true)
 |-- ATMID: string (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- TransactionAmount: double (nullable = true)
 |-- TransactionType: string (nullable = true)
 |-- TransactionTime: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- id: string (nullable = true)
 |-- txn_type: string (nullable = true)
 |-- suspicious_flags: string (nullable = true)
 |-- is_suspicious: integer (nullable = true)
 |-- _rid: string (nullable = true)
 |-- _self: string (nullable = true)
 |-- _etag: string (nullable = true)
 |-- _attachments: string (nullable = true)
 |-- _ts: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- processedAt: string (nullable = true)
 |-- fraud_flags: string (nullable = true)



In [0]:
from pyspark.sql.functions import trim

atm_silver = (
    atm_bronze
    # rename / create clean columns
    .withColumn("transaction_id", col("TransactionID"))
    .withColumn("customer_id",    col("CustomerID"))
    .withColumn("account_number", col("AccountNumber"))
    .withColumn("atm_id",         col("ATMID"))
    .withColumn(
        "amount",
        coalesce(col("TransactionAmount"), col("Amount")).cast("double")
    )
    .withColumn(
        "txn_type",
        coalesce(col("TransactionType"), col("txn_type"))
    )
    .withColumn("status",   col("Status"))
    .withColumn("location", trim(col("Location")))
    .withColumn("channel",  lit("ATM"))
    .withColumn("txn_timestamp", to_timestamp(col("TransactionTime")))
    .withColumn("processed_at",  to_timestamp(col("processedAt")))
    # select only clean columns for silver
    .select(
        "transaction_id",
        "customer_id",
        "account_number",
        "atm_id",
        "amount",
        "txn_type",
        "channel",
        "txn_timestamp",
        "status",
        "location",
        "processed_at",
        "fraud_flags"
    )
)

display(atm_silver.limit(5))
atm_silver.printSchema()


transaction_id,customer_id,account_number,atm_id,amount,txn_type,channel,txn_timestamp,status,location,processed_at,fraud_flags
ATM009778,CUST259,1002003360,ATM035,5000.0,WITHDRAWAL,ATM,2025-01-07T18:57:00Z,SUCCESS,Kolkata,null,null
ATM004496,CUST008,1002003320,ATM020,60000.0,WITHDRAWAL,ATM,2025-01-04T02:55:00Z,SUCCESS,Pune,null,null
ATM004497,CUST183,1002003300,ATM050,45000.0,WITHDRAWAL,ATM,2025-01-04T02:56:00Z,SUCCESS,Hyderabad,null,null
ATM009779,CUST224,1002003496,ATM028,2000.0,WITHDRAWAL,ATM,2025-01-07T18:58:00Z,SUCCESS,Hyderabad,null,null
ATM004498,CUST294,1002003185,ATM001,60000.0,WITHDRAWAL,ATM,2025-01-04T02:57:00Z,SUCCESS,Kolkata,null,null


root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- account_number: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- txn_type: string (nullable = true)
 |-- channel: string (nullable = false)
 |-- txn_timestamp: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- location: string (nullable = true)
 |-- processed_at: timestamp (nullable = true)
 |-- fraud_flags: string (nullable = true)



In [0]:
# Read UPI Bronze
upi_bronze = spark.read.format("delta").load(bronze_upi_path)

display(upi_bronze.limit(5))
upi_bronze.printSchema()


EventID,TxnID,CustomerID,AccountNumber,PayerUPI,PayeeUPI,Amount,TxnType,TxnTimestamp,Status,DeviceID,GeoLocation,id,txn_type,processedAt,fraud_flags,_rid,_self,_etag,_attachments,_ts
UPI007169,TXN007169,CUST065,1002003290,user444@upi,store293@upi,5000,CREDIT,2025-01-01T09:57:20Z,SUCCESS,DEV230,"27.6132,81.795",TXN007169,UPI,2025-12-07T19:56:05.248030,List(),jk1hAP-WDzgBHAAAAAAAAA==,dbs/jk1hAA==/colls/jk1hAP-WDzg=/docs/jk1hAP-WDzgBHAAAAAAAAA==/,"""000007e2-0000-5900-0000-6935dbd50000""",attachments/,1765137365
UPI007170,TXN007170,CUST372,1002003354,user447@upi,store434@upi,75000,CREDIT,2025-01-01T09:57:25Z,SUCCESS,DEV156,"20.2462,77.0446",TXN007170,UPI,2025-12-07T19:56:05.357403,"List(High-value transaction, Unusual UPI Transfer)",jk1hAP-WDzgCHAAAAAAAAA==,dbs/jk1hAA==/colls/jk1hAP-WDzg=/docs/jk1hAP-WDzgCHAAAAAAAAA==/,"""00000be2-0000-5900-0000-6935dbd50000""",attachments/,1765137365
UPI007171,TXN007171,CUST275,1002003438,user333@upi,store496@upi,999,DEBIT,2025-01-01T09:57:30Z,SUCCESS,DEV224,"15.3597,72.7228",TXN007171,UPI,2025-12-07T19:56:05.878828,List(),jk1hAP-WDzgDHAAAAAAAAA==,dbs/jk1hAA==/colls/jk1hAP-WDzg=/docs/jk1hAP-WDzgDHAAAAAAAAA==/,"""000015e2-0000-5900-0000-6935dbd50000""",attachments/,1765137365
UPI007172,TXN007172,CUST369,1002003221,user34@upi,store188@upi,150,CREDIT,2025-01-01T09:57:35Z,SUCCESS,DEV235,"19.9041,86.1759",TXN007172,UPI,2025-12-07T19:56:05.983853,List(),jk1hAP-WDzgEHAAAAAAAAA==,dbs/jk1hAA==/colls/jk1hAP-WDzg=/docs/jk1hAP-WDzgEHAAAAAAAAA==/,"""00001be2-0000-5900-0000-6935dbd50000""",attachments/,1765137365
UPI007173,TXN007173,CUST339,1002003334,user288@upi,store384@upi,5000,DEBIT,2025-01-01T09:57:40Z,SUCCESS,DEV165,"15.5684,85.4305",TXN007173,UPI,2025-12-07T19:56:06.120538,List(),jk1hAP-WDzgFHAAAAAAAAA==,dbs/jk1hAA==/colls/jk1hAP-WDzg=/docs/jk1hAP-WDzgFHAAAAAAAAA==/,"""00001de2-0000-5900-0000-6935dbd60000""",attachments/,1765137366


root
 |-- EventID: string (nullable = true)
 |-- TxnID: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- AccountNumber: long (nullable = true)
 |-- PayerUPI: string (nullable = true)
 |-- PayeeUPI: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- TxnType: string (nullable = true)
 |-- TxnTimestamp: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- DeviceID: string (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- id: string (nullable = true)
 |-- txn_type: string (nullable = true)
 |-- processedAt: string (nullable = true)
 |-- fraud_flags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- _rid: string (nullable = true)
 |-- _self: string (nullable = true)
 |-- _etag: string (nullable = true)
 |-- _attachments: string (nullable = true)
 |-- _ts: long (nullable = true)



In [0]:
upi_silver = (
    upi_bronze
    .withColumn("transaction_id", col("TxnID"))
    .withColumn("event_id",       col("EventID"))
    .withColumn("customer_id",    col("CustomerID"))
    .withColumn("account_number", col("AccountNumber"))
    .withColumn("payer_upi",      col("PayerUPI"))
    .withColumn("payee_upi",      col("PayeeUPI"))
    .withColumn("device_id",      col("DeviceID"))
    .withColumn("location",       col("GeoLocation"))
    .withColumn("amount",         col("Amount").cast("double"))
    .withColumn("txn_type",       coalesce(col("TxnType"), col("txn_type")))
    .withColumn("status",         col("Status"))
    .withColumn("channel",        lit("UPI"))
    .withColumn("txn_timestamp",  to_timestamp(col("TxnTimestamp")))
    .withColumn("processed_at",   to_timestamp(col("processedAt")))
    .select(
        "transaction_id",
        "event_id",
        "customer_id",
        "account_number",
        "amount",
        "txn_type",
        "channel",
        "txn_timestamp",
        "status",
        "payer_upi",
        "payee_upi",
        "device_id",
        "location",
        "processed_at",
        "fraud_flags"
    )
)

display(upi_silver.limit(5))
upi_silver.printSchema()


transaction_id,event_id,customer_id,account_number,amount,txn_type,channel,txn_timestamp,status,payer_upi,payee_upi,device_id,location,processed_at,fraud_flags
TXN007169,UPI007169,CUST065,1002003290,5000.0,CREDIT,UPI,2025-01-01T09:57:20Z,SUCCESS,user444@upi,store293@upi,DEV230,"27.6132,81.795",2025-12-07T19:56:05.24803Z,List()
TXN007170,UPI007170,CUST372,1002003354,75000.0,CREDIT,UPI,2025-01-01T09:57:25Z,SUCCESS,user447@upi,store434@upi,DEV156,"20.2462,77.0446",2025-12-07T19:56:05.357403Z,"List(High-value transaction, Unusual UPI Transfer)"
TXN007171,UPI007171,CUST275,1002003438,999.0,DEBIT,UPI,2025-01-01T09:57:30Z,SUCCESS,user333@upi,store496@upi,DEV224,"15.3597,72.7228",2025-12-07T19:56:05.878828Z,List()
TXN007172,UPI007172,CUST369,1002003221,150.0,CREDIT,UPI,2025-01-01T09:57:35Z,SUCCESS,user34@upi,store188@upi,DEV235,"19.9041,86.1759",2025-12-07T19:56:05.983853Z,List()
TXN007173,UPI007173,CUST339,1002003334,5000.0,DEBIT,UPI,2025-01-01T09:57:40Z,SUCCESS,user288@upi,store384@upi,DEV165,"15.5684,85.4305",2025-12-07T19:56:06.120538Z,List()


root
 |-- transaction_id: string (nullable = true)
 |-- event_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- account_number: long (nullable = true)
 |-- amount: double (nullable = true)
 |-- txn_type: string (nullable = true)
 |-- channel: string (nullable = false)
 |-- txn_timestamp: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- payer_upi: string (nullable = true)
 |-- payee_upi: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- location: string (nullable = true)
 |-- processed_at: timestamp (nullable = true)
 |-- fraud_flags: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
# Read Fraud Bronze
fraud_bronze = spark.read.format("delta").load(bronze_fraud_path)

display(fraud_bronze.limit(5))
fraud_bronze.printSchema()


id,alertType,txn_id,amount,txnType,sourceFile,alertTime,_rid,_self,_etag,_attachments,_ts
TXN003110_High-value transaction,High-value transaction,TXN003110,75000,UPI,upi/upi_transactions.csv,2025-12-07T19:57:25.167603,jk1hALUHT3EBGAAAAAAAAA==,dbs/jk1hAA==/colls/jk1hALUHT3E=/docs/jk1hALUHT3EBGAAAAAAAAA==/,"""11001b8a-0000-5900-0000-6935dc250000""",attachments/,1765137445
TXN003110_Unusual UPI Transfer,Unusual UPI Transfer,TXN003110,75000,UPI,upi/upi_transactions.csv,2025-12-07T19:57:25.276230,jk1hALUHT3ECGAAAAAAAAA==,dbs/jk1hAA==/colls/jk1hALUHT3E=/docs/jk1hALUHT3ECGAAAAAAAAA==/,"""11001e8a-0000-5900-0000-6935dc250000""",attachments/,1765137445
TXN003111_High-value transaction,High-value transaction,TXN003111,75000,UPI,upi/upi_transactions.csv,2025-12-07T19:57:25.473198,jk1hALUHT3EDGAAAAAAAAA==,dbs/jk1hAA==/colls/jk1hALUHT3E=/docs/jk1hALUHT3EDGAAAAAAAAA==/,"""1100248a-0000-5900-0000-6935dc250000""",attachments/,1765137445
TXN003111_Unusual UPI Transfer,Unusual UPI Transfer,TXN003111,75000,UPI,upi/upi_transactions.csv,2025-12-07T19:57:25.581799,jk1hALUHT3EEGAAAAAAAAA==,dbs/jk1hAA==/colls/jk1hALUHT3E=/docs/jk1hALUHT3EEGAAAAAAAAA==/,"""1100278a-0000-5900-0000-6935dc250000""",attachments/,1765137445
TXN003116_High-value transaction,High-value transaction,TXN003116,95000,UPI,upi/upi_transactions.csv,2025-12-07T19:57:26.140486,jk1hALUHT3EFGAAAAAAAAA==,dbs/jk1hAA==/colls/jk1hALUHT3E=/docs/jk1hALUHT3EFGAAAAAAAAA==/,"""1100418a-0000-5900-0000-6935dc260000""",attachments/,1765137446


root
 |-- id: string (nullable = true)
 |-- alertType: string (nullable = true)
 |-- txn_id: string (nullable = true)
 |-- amount: long (nullable = true)
 |-- txnType: string (nullable = true)
 |-- sourceFile: string (nullable = true)
 |-- alertTime: string (nullable = true)
 |-- _rid: string (nullable = true)
 |-- _self: string (nullable = true)
 |-- _etag: string (nullable = true)
 |-- _attachments: string (nullable = true)
 |-- _ts: long (nullable = true)



In [0]:
fraud_silver = (
    fraud_bronze
    .withColumn("alert_id",    col("id"))
    .withColumn("alert_type",  col("alertType"))
    .withColumn("txn_id",      col("txn_id"))
    .withColumn("amount",      col("amount").cast("double"))
    .withColumn("txn_type",    col("txnType"))
    .withColumn("source_file", col("sourceFile"))
    .withColumn("alert_time",  to_timestamp(col("alertTime")))
    .select(
        "alert_id",
        "alert_type",
        "txn_id",
        "amount",
        "txn_type",
        "source_file",
        "alert_time"
    )
)

display(fraud_silver.limit(5))
fraud_silver.printSchema()


alert_id,alert_type,txn_id,amount,txn_type,source_file,alert_time
TXN003110_High-value transaction,High-value transaction,TXN003110,75000.0,UPI,upi/upi_transactions.csv,2025-12-07T19:57:25.167603Z
TXN003110_Unusual UPI Transfer,Unusual UPI Transfer,TXN003110,75000.0,UPI,upi/upi_transactions.csv,2025-12-07T19:57:25.27623Z
TXN003111_High-value transaction,High-value transaction,TXN003111,75000.0,UPI,upi/upi_transactions.csv,2025-12-07T19:57:25.473198Z
TXN003111_Unusual UPI Transfer,Unusual UPI Transfer,TXN003111,75000.0,UPI,upi/upi_transactions.csv,2025-12-07T19:57:25.581799Z
TXN003116_High-value transaction,High-value transaction,TXN003116,95000.0,UPI,upi/upi_transactions.csv,2025-12-07T19:57:26.140486Z


root
 |-- alert_id: string (nullable = true)
 |-- alert_type: string (nullable = true)
 |-- txn_id: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- txn_type: string (nullable = true)
 |-- source_file: string (nullable = true)
 |-- alert_time: timestamp (nullable = true)



In [0]:
display(dbutils.fs.ls("/mnt"))


path,name,size,modificationTime
dbfs:/mnt/bronze/,bronze/,0,0
dbfs:/mnt/gold/,gold/,0,1765188478000
dbfs:/mnt/silver/,silver/,0,1765186728000


In [0]:
import requests

storage_key = "<YOUR_STORAGE_ACCOUNT_KEY_HERE>"

workspace_url = "<databricks_workspace_url>"
token = "<YOUR_DATABRICKS_TOKEN_HERE"

response = requests.post(
    workspace_url + "/api/2.0/secrets/put",
    headers={
        "Authorization": f"Bearer {token}"
    },
    json={
        "scope": "bankscope",
        "key": "storagekey",
        "string_value": storage_key
    }
)

print(response.text)

{}


In [0]:
my_secret = dbutils.secrets.get("bankscope", "storagekey")
print(my_secret)


[REDACTED]


In [0]:
storage_account = "azurebankstorage01"
container = "silver"

dbutils.fs.mount(
    source=f"wasbs://{container}@{storage_account}.blob.core.windows.net",
    mount_point="/mnt/silver",
    extra_configs={
        f"fs.azure.account.key.{storage_account}.blob.core.windows.net": dbutils.secrets.get("bankscope", "storagekey")
    }
)



True

In [0]:
display(dbutils.fs.mounts())


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/mnt/silver,wasbs://silver@azurebankstorage01.blob.core.windows.net,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/bronze,wasbs://bronze@azurebankstorage01.blob.core.windows.net,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,


In [0]:
atm_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .save(silver_atm_path)

print(f"✅ ATM Silver written to: {silver_atm_path}")


✅ ATM Silver written to: /mnt/silver/ATMTransactions


In [0]:
upi_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .save(silver_upi_path)

print(f"✅ UPI Silver written to: {silver_upi_path}")


✅ UPI Silver written to: /mnt/silver/UPIEvents


In [0]:
fraud_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .save(silver_fraud_path)

print(f"✅ Fraud Silver written to: {silver_fraud_path}")


✅ Fraud Silver written to: /mnt/silver/FraudAlerts


In [0]:
display(dbutils.fs.ls("/mnt/silver"))


path,name,size,modificationTime
dbfs:/mnt/silver/ATMTransactions/,ATMTransactions/,0,0
dbfs:/mnt/silver/FraudAlerts/,FraudAlerts/,0,0
dbfs:/mnt/silver/UPIEvents/,UPIEvents/,0,0
